**INSTALL, IMPORT, DOWNLOAD NECESSAY LIBRARY**

In [ ]:
# install & download necessary library not in google collab
!pip install Sastrawi tweepy textblob flair

In [ ]:
# download stopword
nltk.download('stopwords')
# download punkt
nltk.download('punkt')
# download averaged_perceptron_tagger
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [ ]:
# import necessary library
import pandas as pd
import re, string, nltk, Sastrawi, tweepy
from string import punctuation
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

**SCRAPING DATA SOURCE**

In [ ]:
# token access
api_key = "GM524jNn64bT8fn4EVYH3Q0JH"
api_secret_key = "QiCpl8wXVxlT9MIodOxS8nmidPLR2u3XUTWd66udtKi5ltWy20"
bearer_token = "AAAAAAAAAAAAAAAAAAAAAPS4RwEAAAAAaYqNct6cFsghPZjcnbdOuhu1iTM%3Dtk6gaNnsbb4lltbIEv1KJLh9ME9oY1IgBnyAbXb9xAH0qMOjgd"
access_token = "63686818-01YkPafqCKGAPElEzw0CY4tVErMQaBfvcQOElMfJm"
access_token_secret = "hb3hwV8do4cKSqH2Fllj7lCeRbCRBSUyvT1LiGcfQ5u9y"

In [ ]:
# authentication
auth = tweepy.OAuthHandler(api_key, api_secret_key)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

In [ ]:
# Normalize Json to Dataframe
hasilSearch = api.search(q="PPKM", lang="en", count=100)
json_data = [r._json for r in hasilSearch]
df = pd.json_normalize(json_data)
tw_df = df.copy()
tw_df.rename(columns={'text': 'tweet'}, inplace=True)
tw_df.head(3)

,created_at,id,id_str,tweet,truncated,source,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,in_reply_to_screen_name,geo,coordinates,place,contributors,is_quote_status,retweet_count,favorite_count,favorited,retweeted,lang,entities.hashtags,entities.symbols,entities.user_mentions,entities.urls,metadata.iso_language_code,metadata.result_type,user.id,user.id_str,user.name,user.screen_name,user.location,user.description,user.url,user.entities.description.urls,user.protected,user.followers_count,user.friends_count,user.listed_count,user.created_at,...,quoted_status.user.utc_offset,quoted_status.user.time_zone,quoted_status.user.geo_enabled,quoted_status.user.verified,quoted_status.user.statuses_count,quoted_status.user.lang,quoted_status.user.contributors_enabled,quoted_status.user.is_translator,quoted_status.user.is_translation_enabled,quoted_status.user.profile_background_color,quoted_status.user.profile_background_image_url,quoted_status.user.profile_background_image_url_https,quoted_status.user.profile_background_tile,quoted_status.user.profile_image_url,quoted_status.user.profile_image_url_https,quoted_status.user.profile_banner_url,quoted_status.user.profile_link_color,quoted_status.user.profile_sidebar_border_color,quoted_status.user.profile_sidebar_fill_color,quoted_status.user.profile_text_color,quoted_status.user.profile_use_background_image,quoted_status.user.has_extended_profile,quoted_status.user.default_profile,quoted_status.user.default_profile_image,quoted_status.user.following,quoted_status.user.follow_request_sent,quoted_status.user.notifications,quoted_status.user.translator_type,quoted_status.user.withheld_in_countries,quoted_status.geo,quoted_status.coordinates,quoted_status.place,quoted_status.contributors,quoted_status.is_quote_status,quoted_status.retweet_count,quoted_status.favorite_count,quoted_status.favorited,quoted_status.retweeted,quoted_status.possibly_sensitive,quoted_status.lang
0,Fri Jul 23 08:32:52 +0000 2021,1418489277504114688,1418489277504114688,after ppkm what next?,False,"<a href=""https://mobile.twitter.com"" rel=""nofo...",NaN,None,NaN,None,None,None,None,NaN,None,False,0,0,False,False,en,[],[],[],[],en,recent,709820504,709820504,Yangcratears,joshhamn666,bali,everything suck,None,[],False,4,84,0,Sun Jul 22 00:39:52 +0000 2012,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Fri Jul 23 08:30:08 +0000 2021,1418488589998977026,1418488589998977026,Promo Netflix &amp; Viu Premium ‼️\n\nPaket PP...,True,"<a href=""http://twitter.com/download/iphone"" r...",NaN,None,NaN,None,None,None,None,NaN,None,False,0,0,False,False,en,[],[],[],"[{'url': 'https://t.co/GbSrwhscPj', 'expanded_...",en,recent,975268386610409472,975268386610409472,nyanya🍀 jual diamond ml,minyoww,🕗 09:00-21.30,fan/ba acc but sometimes personal acc🍒,https://t.co/cfa5fys8o5,[],False,172,223,0,Sun Mar 18 07:11:17 +0000 2018,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Fri Jul 23 08:29:19 +0000 2021,1418488385153290242,1418488385153290242,RT @medicaIiens: the fact that PSBB was 💗✨🌈😜💫🧃...,False,"<a href=""http://twitter.com/download/android"" ...",NaN,None,NaN,None,None,None,None,NaN,None,False,7,0,False,False,en,[],[],"[{'screen_name': 'medicaIiens', 'name': 'sheal...",[],en,recent,1116645036970500096,1116645036970500096,-,imabadliarhehe,"Bogor, Jawa Barat",i really did talk to god about u!,https://t.co/5U30pfHLIf,[],False,1131,1741,1,Fri Apr 12 10:11:37 +0000 2019,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# jika menggunakan file CSV
#import data from covid-sentiment.csv
#tw_df = pd.read_csv('https://github.com/hpraditya/NLP-IYKRA/blob/main/covid-sentiment.csv')
#tw_df.head(3)

**PRE PROCESSING**

1.   Case folding
2.   Non alphanumeric characters
3.   Stopword removing


In [ ]:
#preprocessing tweet data

#making all letter lowercase
tweet = [x.lower() for x in  tw_df['tweet']]

#remove url
#all pattern start from http and end in whitspace will be removed
tweet = [re.sub(r'http\S+'," ",x) for x in tweet]

#remove twitter pic
#all pattern start from pic.twitter.com and end in whitspace will be removed
tweet = [re.sub(r'pic.twitter.com\S+'," ",x) for x in tweet]

#remove whitespace
tweet = [x.strip() for x in  tweet]

In [ ]:
# remove punctuation and tokenization in one go
#all character from a-Z and 0-9 will not be removed
punc_filter = nltk.RegexpTokenizer(r"\w+")

tweet = [punc_filter.tokenize(x) for x in tweet]

tweet[0]

['after', 'ppkm', 'what', 'next']

In [ ]:
#collect the stopwords 
stop_words = stopwords.words('indonesian')
factory = StemmerFactory()
stemmer = factory.create_stemmer()


#stopwrods removal 
nonstop_words = []
for stc in  tweet:
    words = [x for x in stc if x not in stop_words]
    
    nonstop_words.append(words)

nonstop_words[0]

# rejoin the list of words to make sentences
tweet = nonstop_words
text_tweet = [" ".join(x) for x in tweet]
text_tweet[0]


'after ppkm what next'

**Feature Engineering**
1.   Bag of words
2.   Tf-Idf techniques


In [ ]:
# make bag of words
bow = CountVectorizer()

#fit text_tweet to bow
x_sentence = bow.fit_transform(text_tweet)

#get feature names from bow
x_sentence_df = pd.DataFrame(x_sentence.toarray(),columns=bow.get_feature_names())
x_sentence_df

,10,100,12am,167378399261619,19,1975,20,2021,24,25,27,30,40,50k,about,ac,accept,act,activity,added,after,age,agriculture,air,all,als,always,amid,amp,anakodok2009,and,another,apparently,aq,are,argue,around,arrange,art,as,...,vampmotionless,ve,viu,walk,walks,wanna,want,was,waspada,we,wedding,week,weeks,well,were,wfh,what,when,whenever,whether,whitout,who,will,with,without,wondering,world,worse,would,wtf,yeah,year,yesterday,yoas,you,your,yourself,yuk,yusufgunawan,zarazettirazr
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,...,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
96,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
97,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
98,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
#making tfidf vectorizer
tfidf = TfidfVectorizer()

#create new list with fewer sentence so memory will fit
text_tweet2 = text_tweet[0:15000]

#fit text_tweet2 to tfidf
x2_sentence = tfidf.fit_transform(text_tweet2)

#get frequency count of each feature names
x2_sentence_df = pd.DataFrame(x2_sentence.toarray(),columns=tfidf.get_feature_names())
x2_sentence_df.head(5)

,10,100,12am,167378399261619,19,1975,20,2021,24,25,27,30,40,50k,about,ac,accept,act,activity,added,after,age,agriculture,air,all,als,always,amid,amp,anakodok2009,and,another,apparently,aq,are,argue,around,arrange,art,as,...,vampmotionless,ve,viu,walk,walks,wanna,want,was,waspada,we,wedding,week,weeks,well,were,wfh,what,when,whenever,whether,whitout,who,will,with,without,wondering,world,worse,would,wtf,yeah,year,yesterday,yoas,you,your,yourself,yuk,yusufgunawan,zarazettirazr
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.569279,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.569279,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.238604,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.223406,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.477208,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.245063,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.323226,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.645722,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.245063,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.323226,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


**SUMMARY POS-TAGGING and NER**

POS-TAGGING atau Part of Speech Tagging adalah metode untuk mengindentifikasi struktur kalimat. Beberapa library yang digunakan Spacy, Flair, CRFTagger, UUParser, Syntaxnet, StanfordNLP. Adapun fungsi atau kegunaan dari POS-TAGGING adalah sebagai berikut:

*   Mengidentifikasi makna sebuah kata dalam kalimat
*   Mengklasifikasikan kata berdasarkan komponen struktur kalimat
*   Meminimalisir ambiguitas dalam kalimat/frase

POS-Tagging terbagi menjadi dua tipe :
1. Rule-Based POS tagging
2. Stochastic POS Tagging

Rule-Based POS Tagging melakukan tagging berdasarkan aturan yang dibuat berdasarkan informasi kontekstual. Penentuan makna kata pada Rule-Based POS Tagging biasa dilakukan melihat fitur linguistik kata, kata sebelumnya, dan kata sesudahnya.Contohnya untuk bahasa inggris : jika kata sebelumnya adalah sebuah article, maka kata tsb. adalah sebuah noun. Pembuatan rule ini tidak perlu secara manual, feature engineering yang dilakukan adalah membuat template rule untuk model. Contoh metode yang digunakan dalam rule-based POS Tagging adalah Brill's Tagger.

Stochastic POS Tagging melakukan tagging berdasarkan probability. Probability ini bisa digunakan dari frekuensi sebuah kata dalam suatu corpus atau dari probability kemunculan sebuah kata berdasarkan urutan kata-kata sebelumnya. Bisa juga dengan menggabungkan word frequency measurement dan dan tag sequence probability. Salah satu  metode yang digunakan dalam Stochastic POS tagging adalah Hidden Markov Model

Beberapa aplikasi untuk POS Tagging adalah sebagai berikut :
1. Speech Recognition
2. Machine Translation
3. Question Answering by bot

NER atau Named Entity Recognition adalah metode untuk mengidentifikasi entitas seperti lokasi, organisasi, orang, atau entitas lain yang ditentukan dari dokumen atau data teks. Beberapa contoh library yang bisa digunakan adalah Spacy, Flair, Mbert (Multilingual), dan camembert (French). Adapun fungsi atau kegunaan dari NER adalah sebagai berikut:

*   Klasifikasi dan identifikasi suatu dokumen atau teks
*   Memudahkan identifikasi berdasarkan jumlah kemunculan entitas pada teks sehingga lebih cepat dan akurat





### NER

In [ ]:
from flair.data import Sentence
from flair.models import SequenceTagger

In [ ]:
text = text_tweet[99]
type(text)

str

In [ ]:
text = Sentence(text)
type(text)

flair.data.Sentence

In [ ]:
tagger = SequenceTagger.load('ner') # Load the tagger
tagger.predict(text) # Run NER in the text

2021-07-23 08:34:50,750 --------------------------------------------------------------------------------
2021-07-23 08:34:50,752 The model key 'ner' now maps to 'https://huggingface.co/flair/ner-english' on the HuggingFace ModelHub
2021-07-23 08:34:50,754  - The most current version of the model is automatically downloaded from there.
2021-07-23 08:34:50,755  - (you can alternatively manually download the original model at https://nlp.informatik.hu-berlin.de/resources/models/ner/en-ner-conll03-v0.4.pt)
2021-07-23 08:34:50,757 --------------------------------------------------------------------------------
2021-07-23 08:34:50,812 loading file /root/.flair/models/ner-english/4f4cdab26f24cb98b732b389e6cebc646c36f54cfd6e0b7d3b90b25656e4262f.8baa8ae8795f4df80b28e7f7b61d788ecbb057d1dc85aacb316f1bd02837a4a4


In [ ]:
print(text) 

Sentence: "lol get themmm ive seen ppl complain about this ppkm and shit but they don t even do their part to social distance"   [− Tokens: 23]


In [ ]:
for entity in text.get_spans('ner'):
    print(entity)

### POS-Tagging


In [ ]:
text_tweet[99]

'lol get themmm ive seen ppl complain about this ppkm and shit but they don t even do their part to social distance'

In [ ]:
tokenize = nltk.word_tokenize(text_tweet[99]) # Tokenize the word
tokenize

['lol',
 'get',
 'themmm',
 'ive',
 'seen',
 'ppl',
 'complain',
 'about',
 'this',
 'ppkm',
 'and',
 'shit',
 'but',
 'they',
 'don',
 't',
 'even',
 'do',
 'their',
 'part',
 'to',
 'social',
 'distance']

In [ ]:
pos_tag = nltk.pos_tag(tokenize) # Apply the pos_tag
pos_tag

[('lol', 'NN'),
 ('get', 'VB'),
 ('themmm', 'JJ'),
 ('ive', 'JJ'),
 ('seen', 'VBN'),
 ('ppl', 'NN'),
 ('complain', 'NN'),
 ('about', 'IN'),
 ('this', 'DT'),
 ('ppkm', 'NN'),
 ('and', 'CC'),
 ('shit', 'NN'),
 ('but', 'CC'),
 ('they', 'PRP'),
 ('don', 'VBP'),
 ('t', 'JJ'),
 ('even', 'RB'),
 ('do', 'VBP'),
 ('their', 'PRP$'),
 ('part', 'NN'),
 ('to', 'TO'),
 ('social', 'JJ'),
 ('distance', 'NN')]